# Compute performance metrics for the given Y and Y_score without sklearn

In [ ]:
# Mount to drive to access the files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
# other than these two we should not import any other packages


## A. Compute performance metrics for the given data '5_a.csv'
 <pre>  <b>Note 1:</b> in this data you can see number of positive points >> number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_a.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a> Note: it should be numpy.trapz(tpr_array, fpr_array) not numpy.trapz(fpr_array, tpr_array)
Note- Make sure that you arrange your probability scores in descending order while calculating AUC</li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [ ]:
# Reading data from csv into a dataframe
df_a=pd.read_csv('/content/drive/MyDrive/AAIC_Assignments/ML/AS5/5_a.csv')
df_a.head

<bound method NDFrame.head of          y     proba
0      1.0  0.637387
1      1.0  0.635165
2      1.0  0.766586
3      1.0  0.724564
4      1.0  0.889199
...    ...       ...
10095  1.0  0.665371
10096  1.0  0.607961
10097  1.0  0.777724
10098  1.0  0.846036
10099  1.0  0.679507

[10100 rows x 2 columns]>

In [ ]:
# write your code here for task A
print(df_a.head(5))  # The data set is baised towards positive points(1)
print("\n",df_a.shape)

     y     proba
0  1.0  0.637387
1  1.0  0.635165
2  1.0  0.766586
3  1.0  0.724564
4  1.0  0.889199

 (10100, 2)


In [ ]:
#print(df_a['proba'])

In [ ]:
# Function to predict y
def predict_y(df,proba,thresh_hold):
    y_pred=[]
    for c_label_prob in df[proba]:
        if c_label_prob<thresh_hold:
            y_pred.append(0)
        else:
            y_pred.append(1)
    return y_pred        

In [ ]:
# Fucntion to computer confusion matrix.
def confusion_matrix_values(df,y,y_pred):
    TP = 0 # True positive 
    TN = 0 # True Negative 
    FN = 0 # False Positive
    FP = 0 # False Negative
    
    for val1, val2 in enumerate(df['y']):
        #print(val1,val2)
        if (df.y_pred[val1]==1) and (df.y[val1]==1):
            TP += 1
        if (df.y_pred[val1]==0) and (df.y[val1]==0):
            TN += 1
            
        if (df.y_pred[val1]==0) and (df.y[val1]==1):
            FN += 1
            
        if (df.y_pred[val1]==1) and (df.y[val1]==0):
            FP += 1
    return {'TN':TN,'TP':TP,'FN':FN,'FP':FP}            
            
               

In [ ]:
# As given in the problem statement of A.
thresh_hold = 0.5
df_a['y_pred'] = predict_y(df_a,'proba',thresh_hold)
confusion_matrix = confusion_matrix_values(df_a,'y','y_pred')

In [ ]:
# Display Confusion matrix
print('Confusion Matrix : ', confusion_matrix)

Confusion Matrix :  {'TP': 10000, 'TN': 0, 'FN': 0, 'FP': 100}


In [ ]:
# F-1  Score
'''
precsion = true postive / true postive + false postive
recall   = true positive / positive          # positive = true positive + false negative  
F-1 Score = 2 * (precison * recall) / (precision + recall)
'''
# positive = true positive + false negative 
total_points = df_a.y.value_counts()
#print(total_points)
positive = total_points[1]  # filtering only postive class label '1' points
#print(positive)

# Precision
precision = confusion_matrix['TP'] / (confusion_matrix['TP'] + confusion_matrix['FP'])
#print(precision)

# Recall
recall = confusion_matrix['TP'] / positive


# F-1 Score = 2 * (precison * recall) / (precision + recall)

F1_score =2*precision*recall/(precision+recall)

# Display F-1 Score
print('F-1 Score is : ',F1_score)

F-1 Score is :  0.9950248756218906


In [ ]:
# Shape of data set
print(df_a.shape)
print(type(df_a.shape))

(10100, 3)
<class 'tuple'>


In [ ]:
# Accuracy
'''
Accuracy = Total no.of correctly classified points / total number of points in Test set

Total no.of correctly classified points = true positives + true negatives
'''
total_points_in_Dtest_set = df_a.shape[0]
accuracy = (confusion_matrix['TP'] + confusion_matrix['TN']) / total_points_in_Dtest_set 

# Display Accuaracy 
print('Accuracy is : ',accuracy)

Accuracy is :  0.9900990099009901


In [ ]:
# Total points in the dataset
tot_pts = df_a.y.value_counts()
tot_pts

1.0    10000
0.0      100
Name: y, dtype: int64

In [ ]:
print(tot_pts)

1.0    10000
0.0      100
Name: y, dtype: int64


In [ ]:
print(tot_pts[0])

100


In [ ]:
print(tot_pts[1])

10000


In [ ]:
# Function to compute AUC score

# for interations visualization we are importing tqdm
from tqdm import tqdm
# While computing AUC score we need to calculate "TPR,"FPR" at every threshold by using 
# actual "y" and predicted "y_pred".

def acu_score(df):
  tot_pts = df_a['y'].value_counts()
  P = tot_pts[1]
  N = tot_pts[0]
  TPR = []
  FPR = []

  # We consider each predicted probability value as threshhold value
  # for each threshhold value 
  for i in tqdm(df['proba']):
    df['y_pred'] = predict_y(df,'proba',i)
    confusion_matrix = confusion_matrix_values(df,'y','y_pred')
    TPR.append(confusion_matrix['TP']/P)
    FPR.append(confusion_matrix['FP']/N)
    df.drop(columns=['y_pred'])

  return np.trapz(TPR, FPR)  
  #return np.trapz(FPR, TPR) 


In [ ]:
df_a.shape

(10100, 3)

In [ ]:
df_a.head()

,y,proba,y_pred
0,1.0,0.637387,1
1,1.0,0.635165,1
2,1.0,0.766586,1
3,1.0,0.724564,1
4,1.0,0.889199,1


In [ ]:
# Sort the values in decesnding order as the AUC doesn't depends on the scores 
# and it only depends on the order of values
data_auc = df_a.sort_values(by='proba',ascending=False) # sort in descending order

# We don't need y_predicted value to compute AUC score. So, we drop that column
data_auc=data_auc.drop(columns=['y_pred'])

In [ ]:
data_auc.shape

(10100, 2)

In [ ]:
# Display AUC Score
AUC_Score = acu_score(data_auc)
#print('AUC Score is : ',AUC_Score)

100%|██████████| 10100/10100 [1:45:01<00:00,  1.60it/s]


In [ ]:
print('AUC Score for problem A is : ',AUC_Score)

AUC Score for problem A is :  0.48829900000000004




## B. Compute performance metrics for the given data '5_b.csv'
<pre>
   <b>Note 1:</b> in this data you can see number of positive points << number of negatives points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_b.csv</b>
   <b>Note 3:</b> you need to derive the class labels from given score</pre> $y^{pred}= \text{[0 if y_score < 0.5 else 1]}$

<pre>
<ol>
<li> Compute Confusion Matrix </li>
<li> Compute F1 Score </li>
<li> Compute AUC Score, you need to compute different thresholds and for each threshold compute tpr,fpr and then use               numpy.trapz(tpr_array, fpr_array) <a href='https://stackoverflow.com/q/53603376/4084039'>https://stackoverflow.com/q/53603376/4084039</a>, <a href='https://stackoverflow.com/a/39678975/4084039'>https://stackoverflow.com/a/39678975/4084039</a>
Note- Make sure that you arrange your probability scores in descending order while calculating AUC</li>
<li> Compute Accuracy Score </li>
</ol>
</pre>

In [ ]:
df_b=pd.read_csv('/content/drive/MyDrive/AAIC_Assignments/ML/AS5/5_b.csv')
df_b.head(5)

,y,proba
0,0.0,0.281035
1,0.0,0.465152
2,0.0,0.352793
3,0.0,0.157818
4,0.0,0.276648


In [ ]:
# See the shaper (rows and columns)
df_b.shape

(10100, 2)

In [ ]:
# write your code here for task B
# As give in the probelm B, thersh_hold =0.5
t = 0.5
df_b['y_pred'] = predict_y(df_b,'proba',t)
B_confusion_matrix = confusion_matrix_values(df_b,'y','y_pred')

# Display confusion matrix 
print('confusion matrix values for problem B is : ',B_confusion_matrix)

confusion matrix values for problem B is :  {'TN': 9761, 'TP': 55, 'FN': 45, 'FP': 239}


In [ ]:
# F-1  Score
'''
precsion = true postive / true postive + false postive
recall   = true positive / positive          # positive = true positive + false negative  
F-1 Score = 2 * (precison * recall) / (precision + recall)
'''
# positive = true positive + false negative 
total_points = df_b.y.value_counts()
positive = total_points[1]  # filtering only postive class label '1' points


# Precision
precision = B_confusion_matrix['TP'] / (B_confusion_matrix['TP'] + B_confusion_matrix['FP'])


# Recall
recall = B_confusion_matrix['TP'] / positive


# F-1 Score = 2 * (precison * recall) / (precision + recall)
F1_score =2*precision*recall/(precision+recall)

# Display F-1 Score
print('F-1 Score for problem B is : ',F1_score)

F-1 Score for problem B is :  0.2791878172588833


In [ ]:
# Sort the values in decesnding order as the AUC doesn't depends on the scores 
# and it only depends on the order of values
data_auc_B = df_b.sort_values(by='proba',ascending=False) # sort in descending order

# We don't need y_predicted value to compute AUC score. So, we drop that column
data_auc_B.drop(columns='y_pred')


# Display AUC Score
AUC_Score_B = acu_score(data_auc_B)
#print('AUC Score is : ',AUC_Score_B)


100%|██████████| 10100/10100 [1:43:50<00:00,  1.62it/s]

AUC Score is :  0.9377570000000001


In [ ]:
print('AUC Score is : ',AUC_Score_B)

AUC Score is :  0.9377570000000001


In [ ]:
# Accuracy score for problem B
# Accuracy
'''
Accuracy = Total no.of correctly classified points / total number of points in Test set

Total no, of correctly classified points = true positives + true negatives
'''
#total_points_in_Dtest = df_a.shape[0]
# shape[0] gives total no.of points in the dataset
accuracy_B = (B_confusion_matrix['TP'] + B_confusion_matrix['TN']) / df_b.shape[0] 

# Display Accuaracy 
print('Accuracy is : ',accuracy_B)

Accuracy is :  0.9718811881188119


### C. Compute the best threshold (similarly to ROC curve computation) of probability which gives lowest values of metric <b>A</b> for the given data 
<br>

you will be predicting label of a data points like this: $y^{pred}= \text{[0 if y_score < threshold  else 1]}$

$ A = 500 \times \text{number of false negative} + 100 \times \text{numebr of false positive}$

<pre>
   <b>Note 1:</b> in this data you can see number of negative points > number of positive points
   <b>Note 2:</b> use pandas or numpy to read the data from <b>5_c.csv</b>
</pre>

In [ ]:
df_c=pd.read_csv('/content/drive/MyDrive/AAIC_Assignments/ML/AS5/5_c.csv')
df_c.head(5)

,y,prob
0,0,0.458521
1,0,0.505037
2,0,0.418652
3,0,0.412057
4,0,0.375579


In [ ]:
df_c.shape

(2852, 2)

In [ ]:
tp = df_c.y.value_counts()
print(tp)

0    1805
1    1047
Name: y, dtype: int64


In [ ]:
print(df_c['prob'])

0       0.458521
1       0.505037
2       0.418652
3       0.412057
4       0.375579
          ...   
2847    0.491663
2848    0.292109
2849    0.659161
2850    0.456265
2851    0.659161
Name: prob, Length: 2852, dtype: float64


In [ ]:
# write your code for task C
# Function to compute minimum(lowest value) for metric A as stated in problem C description

# df should be in sorted in descending order
def min_metric(df):
  tot_pts = df_c.y.value_counts()
  P = tot_pts[1]
  N = tot_pts[0] 
  metric = {}

  for t in tqdm(df['prob']):
    df['y_pred'] = predict_y(df,'prob',t)
    confusion_matrix = confusion_matrix_values(df,'y','y_pred')
    # 500×number of false negative + 100×numebr of false positive
    metric_val = (500 * confusion_matrix['FN']) + (100 * confusion_matrix['FP'])
    metric[t] = metric_val
    df.drop(columns=['y_pred'])
  
  return metric    


In [ ]:
data = df_c.sort_values(by='prob',ascending=False)
d = min_metric(data)

100%|██████████| 2852/2852 [08:29<00:00,  5.59it/s]


In [ ]:
m = min(d.values())
result = [key for key in d if d[key]==m]
print('minimum value of the specified metric is ',result,' : ',m)

minimum value of the specified metric is  [0.2300390278970873]  :  141000



## D.</b></font> Compute performance metrics(for regression) for the given data 5_d.csv
<pre>    <b>Note 2:</b> use pandas or numpy to read the data from <b>5_d.csv</b>
    <b>Note 1:</b> <b>5_d.csv</b> will having two columns Y and predicted_Y both are real valued features
<ol>
<li> Compute Mean Square Error </li>
<li> Compute MAPE: https://www.youtube.com/watch?v=ly6ztgIkUxk</li>
<li> Compute R^2 error: https://en.wikipedia.org/wiki/Coefficient_of_determination#Definitions </li>
</ol>
</pre>

In [ ]:
df_d=pd.read_csv('/content/drive/MyDrive/AAIC_Assignments/ML/AS5/5_d.csv')
df_d.head(5)

,y,pred
0,101.0,100.0
1,120.0,100.0
2,131.0,113.0
3,164.0,125.0
4,154.0,152.0


In [ ]:
 # write your code for task 5d

 # fucntion to compute error
def error(df, col1, col2):
    err = []
    for index, (val1,val2) in enumerate(zip(df[col1],df[col2])):
        err.append(val1-val2)
        
    return err

#result = error(df_d,'y','pred')
#print(result)


# function to compute absolute error
def abs_error(df,col):
    abs_val=[]
    for index, value in enumerate(df[col]):
        abs_val.append(abs(value))
        
    return abs_val


# function for SS_residue 

def ss_res(df,col):
    resi =0
    for index, value in enumerate(df[col]):
        # ss_residue = sum(ei)2
        resi += value**2
        
    return resi

# ss_total = sum(squared difference between y_i and y_bar) 
def ss_tot(df,col):
    tot=0
    mean_val = df['y'].mean()
    for index, value in enumerate(df[col]):
        tot += (value - mean_val)**2
        
    return tot


# Function to compute Mean Square Error
def mean_sqrd_error(df,col):
    mean_sqrd_err = 0
    for index,value in enumerate(df[col]):
        return ss_res(df,col) / len(df[col])
    
    

# Function to compute 'Mean absolute percentage error'

def mape(df,col1,col2):
    mape_val = sum(df[col1]) / sum(df[col2])
    return mape_val
 


In [ ]:
# Adding error column into dataframe
df_d['error']=error(df_d,'y','pred')
df_d['abs_error']=abs_error(df_d,'error')

In [ ]:
# Mean Square Error
MSE = mean_sqrd_error(df_d,'error')
# Display mean squared error
print('Mean Square Error of problem D is : ',MSE)

Mean Square Error of problem D is :  177.16569974554707


In [ ]:
# Mean absolute percentage error : MAPE
MAPE = mape(df_d,'abs_error','y')
# Display 'Mean absolute percentage error'
print('Mean absolute percentage error(MAPE) is : ',MAPE)


Mean absolute percentage error(MAPE) is :  0.1291202994009687


In [ ]:
# Compute R^2 error

SS_RESI = ss_res(df_d,'error')
SS_TOT = ss_tot(df_d,'y')

# R^2 = 1 - (ss_residue / ss_total)
R_Squared = 1 - (SS_RESI / SS_TOT)

# Display R^2 : Coefficient of determination
print('Coefficient of determination or R^2 value is : ',R_Squared)

Coefficient of determination or R^2 value is :  0.9563582786990964
